# Análisis de audio con Python

En este notebook exploraremos los conceptos de audio mono y estéreo y compararemos los datos de cada tipo de audio.

### Preparación inicial

Comenzamos importando las librerías necesarias, entre ellas wavfile para poder procesar nuestro archivo .wav

In [ ]:
from scipy.io import wavfile  # Importa lectura/escritura WAV
import IPython  # Importa utilidades de audio en notebook
import os  # Importa funciones del sistema de archivos
import numpy as np  # Importa operaciones numéricas
import matplotlib.pyplot as plt  # Importa herramientas de gráficas

### Configuración de directorios

Guardaremos los audios de ejemplo que utilizaremos y las salidas de nuestras operaciones en la carpeta audio. Utilizaremos con frecuencia estas dos rutas así que las guardaremos en variables para poder utilizarlas cómodamente.

In [ ]:
cwd = os.getcwd()  # Obtiene el directorio actual
input = os.path.join(cwd, os.path.join('audio', 'examples'))  # Define carpeta de entrada
output = os.path.join(cwd, os.path.join('audio', '_output'))  # Define carpeta de salida

### Lectura del archivo

Tras preparar las rutas, usamos esta información para obtener la ruta del archivo que deseamos, en este caso un audio .wav. Lo abrimos con la librería "os" y extraemos información sobre la frecuencia de muestreo y los datos del audio.

In [ ]:
filename = os.path.join(input, 'breaking_bad.wav')  # Construye la ruta del audio
sample_rate, audio_data = wavfile.read(filename)  # Lee frecuencia y muestras

### Extracción de datos

Ahora que contamos con la estructura que contiene los datos del archivo, podemos extraer sus datos y comprobar la información del audio. Esta estructura es una matriz que en el caso de el audio estéreo contiene información de cada canal en algunas de sus filas.

Podemos obtener también el tamaño del archivo usando de nuevo la librería "os".

In [ ]:
print('Datos de audio (estereo):\n')  # Muestra cabecera informativa
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')  # Muestra frecuencia en kHz
print(f'Número de canales:   {audio_data.shape[1]}')  # Muestra cantidad de canales
print(f'- 1º canal:   {audio_data[:5, 0]}')  # Muestra primeras muestras del canal izquierdo
print(f'- 2º canal:   {audio_data[:5, 1]}')  # Muestra primeras muestras del canal derecho
print(f'Tamaño del archivo: {os.path.getsize(filename) / 1024 ** 2} MB')  # Muestra tamaño del archivo

### Reproducción

Usando las funciones de audio de IPython, podemos reproducir el archivo directamente en el notebook. En este caso estamos usando audio estéreo, por lo que la función necesita la matriz traspuesta del objeto de numpy, además de la tasa de muestreo.

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate)  # Reproduce el audio estéreo

### Conversión a mono

Podemos convertir el audio a mono calculando la media en base al eje 1 del audio estéreo. Tras esta conversión es posible que numpy haya convertido el array de números de 16 bits del archivo wav a uno de 32, por lo que hay que reconvertirlo a números de 16 bits.

Después, escribimos estos nuevos datos a un nuevo archivo en la carpeta output y mostramos los datos del nuevo archivo.

In [ ]:
new_data_mono = audio_data.mean(axis=1)  # Promedia canales para convertir a mono
new_data_mono = new_data_mono.astype(np.int16)  # Convierte al tipo entero de 16 bits
filename_mono=os.path.join(output, 'sample1_mono.wav')  # Define ruta del archivo mono
wavfile.write(  # Inicia escritura del WAV mono
    filename=filename_mono,  # Usa la ruta de salida
    rate=sample_rate,  # Mantiene la frecuencia de muestreo
    data=new_data_mono  # Guarda las muestras mono
)  # Finaliza la escritura del archivo
print('Datos de audio (mono):\n')  # Muestra cabecera del audio mono
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')  # Muestra frecuencia en kHz
print(f'Nuevo tamaño: {new_data_mono.shape}')  # Muestra el tamaño del arreglo mono
print(f'Canal unico:  {new_data_mono[:5]}...')  # Muestra primeras muestras mono
print(f'Número de canales:   {new_data_mono.ndim}')  # Muestra dimensiones del audio mono
print(f'Tamaño del archivo: {os.path.getsize(filename_mono) / 1024 ** 2} MB')  # Muestra tamaño del nuevo archivo

### Reproducción en mono

Reproducimos el archivo resultante y comprobamos que los dos canales suenan igual.

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)  # Reproduce el audio mono

In [ ]:
t_stereo = np.arange(audio_data.shape[0]) / sample_rate  # Crea eje temporal del estéreo
t_mono = np.arange(new_data_mono.shape[0]) / sample_rate  # Crea eje temporal del mono
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)  # Crea figura con dos subgráficas
axes[0].plot(t_stereo, audio_data[:, 0], label='Canal izquierdo', alpha=0.8)  # Grafica canal izquierdo
axes[0].plot(t_stereo, audio_data[:, 1], label='Canal derecho', alpha=0.8)  # Grafica canal derecho
axes[0].set_title('Audio estéreo en el dominio del tiempo')  # Define título del gráfico estéreo
axes[0].set_ylabel('Amplitud')  # Etiqueta el eje Y estéreo
axes[0].legend(loc='upper right')  # Muestra leyenda de canales
axes[0].grid(True, alpha=0.3)  # Activa rejilla suave en estéreo
axes[1].plot(t_mono, new_data_mono, color='tab:green')  # Grafica la señal mono
axes[1].set_title('Audio mono en el dominio del tiempo')  # Define título del gráfico mono
axes[1].set_xlabel('Tiempo (s)')  # Etiqueta el eje X
axes[1].set_ylabel('Amplitud')  # Etiqueta el eje Y mono
axes[1].grid(True, alpha=0.3)  # Activa rejilla suave en mono
plt.tight_layout()  # Ajusta márgenes automáticamente
plt.show()  # Muestra las gráficas

### Diferencias entre estéreo y mono

El audio en estéreo cuenta con datos de audio para cada canal del dispositivo de audio, por ejemplo, para cada auricular de unos cascos para funciones como audio envolvente en una película o videojuego.

El audio mono solo tiene un canal que se reproduce en todos los dispositivos de sonido y ocupan mucho menos espacio debido a que solo tienen que guardar información de un canal.

In [ ]:
n = len(new_data_mono)
Fs = sample_rate

ch_Fourier = np.fft.fft(new_data_mono)  # ch1

abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

In [ ]:
eps = [1e-5, .02, .041, .063, .086, .101, .123]

eps = eps[0]
print(f'Epsilon: {eps}')

thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

spec_energy = np.cumsum(abs_ch_Fourier)

frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

In [ ]:
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

new_data = new_data_mono[::D]

wav_compressed_file = "sample_compressed.wav"

wavfile.write(
    filename=os.path.join(output, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(output, wav_compressed_file))

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

In [ ]:
!ls -sh audio/_output/sample_compressed.wav

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

### URL del repositorio

https://github.com/FranMUex/Audio_SM